# Tasks
- used `kfoldStratified` splitting for this experiment _(`10 fold`)_.

- all the test are done inside the the loop of each fold to prevent data leakage and shuffle was `True`
    - Outliers more than `2xStandard Deviation` have been replaced with `mean`
    - removed `race` and `native-country`
    - Balance the dataset to directly by using `SmoteTomek` by keeping the ratio 1 for majority and minority class.
    - *`No Scaling`* was used to scaling the train and test values
    - Null values for both train and test have been replaced with `mean` of train.
- Highest accuracy: `('LightGBM', 0.8593628391581308)`

# Import Libraries

In [1]:
import pandas as pd
import my_packages as mypckg
from collections import Counter
import numpy as np
from imblearn.combine import SMOTETomek

%matplotlib inline

# Dataset

In [2]:
df = pd.read_csv('../data/adult.csv', na_values="?", skipinitialspace=True)

## Categorical Data Encoding

In [3]:
# encode the dataset
df , mappings = mypckg.encoder(df)

## Feature and Target Separation

In [4]:
X = df.drop(['race', 'native-country', 'income'], axis=1)
y = df['income']

# kfold

In [5]:
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

## Apply Models

In [6]:
models = mypckg.get_models()
models

{'Logistic Regression': LogisticRegression(max_iter=1000),
 'Decision Tree': DecisionTreeClassifier(),
 'Random Forest': RandomForestClassifier(),
 'Support Vector Classifier': SVC(),
 'Naive Bayes': GaussianNB(),
 'Gradient Boosting': GradientBoostingClassifier(),
 'AdaBoost': AdaBoostClassifier(),
 'LightGBM': LGBMClassifier(),
 'XGBoost': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric='logloss',
               feature_types=None, gamma=None, grow_policy=None,
               importance_type=None, interaction_constraints=None,
               learning_rate=None, max_bin=None, max_cat_threshold=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
               max_leaves=None, min_child_weight=None, missing=nan,
               monotone_constraints=None,

In [7]:
model_accuracies = {}
for name, model in models.items():
    fold_number = 1
    accuracies = []
    print(f"{'*' * 100}\n{'=' * 100}\nProcessing model: {name}\n{'=' * 100}\n{'*' * 100}")
    for train_index, test_index in kf.split(X, y):
        print(f"{'*' * 100}\n{':' * 100}\nModel Name: {name}\nFold # {fold_number}\n{':' * 100}\n{'*' * 100}")
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        if X_train.empty or X_test.empty:
            print("Empty train or test set encountered. Skipping this split.")
            continue


        X_train.fillna(X_train.mean(), inplace=True)
        X_test.fillna(X_train.mean(), inplace=True)

        y_train = y_train.loc[X_train.index]
        y_test= y_test.loc[X_test.index]

        for col in X_train.columns:
            mean = X_train[col].mean()
            sd = X_train[col].std()
            X_train[col].mask(X_train[col] > mean+(2*sd), mean, inplace=True)
            X_train[col].mask(X_train[col] < mean-(2*sd), mean, inplace=True)
        
        smote_tomek = SMOTETomek(sampling_strategy=1, random_state=42)
        X_train, y_train = smote_tomek.fit_resample(X_train, y_train)
        print(f"Y after resampling: {Counter(y_train)}")

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracies.append(mypckg.accuracy_score(y_test, y_pred))

        mypckg.print_evaluation_metrics(y_test, y_pred, model)
        fold_number += 1
    average_accuracy = np.mean(accuracies)
    print(f"Average Accuracy for {name}: {average_accuracy}\n\n")
    model_accuracies[name] = average_accuracy
print("All the accuracies:")
print("*" * 100)
print("*" * 100)
mypckg.show(model_accuracies)

****************************************************************************************************
Processing model: Logistic Regression
****************************************************************************************************
****************************************************************************************************
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
Model Name: Logistic Regression
Fold # 1
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
****************************************************************************************************


C:\Users\raqib\AppData\Local\Temp\ipykernel_15988\3388014965.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.fillna(X_train.mean(), inplace=True)
C:\Users\raqib\AppData\Local\Temp\ipykernel_15988\3388014965.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(X_train.mean(), inplace=True)
C:\Users\raqib\AppData\Local\Temp\ipykernel_15988\3388014965.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on 

Y after resampling: Counter({0: 30825, 1: 30825})


e:\codes\data-analysis\adult-income-prediction\env-adult-income-prediction\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\raqib\AppData\Local\Temp\ipykernel_15988\3388014965.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.fillna(X_train.mean(), inplace=True)
C:\Users\raqib\AppData\Local\Temp\ipykernel_15988\3388014965.py:17: SettingWithCopyWarning

Model: LogisticRegression(max_iter=1000)
Accuracy: 0.6577277379733879
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.64      0.74      3716
           1       0.38      0.71      0.50      1169

    accuracy                           0.66      4885
   macro avg       0.63      0.68      0.62      4885
weighted avg       0.76      0.66      0.68      4885

Confusion Matrix:
[[2379 1337]
 [ 335  834]]
--------------------------------------------------------------------------------
****************************************************************************************************
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
Model Name: Logistic Regression
Fold # 2
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
****************************************************************************************************
Y after r

...

Y after resampling: Counter({0: 30963, 1: 30963})
Model: BaggingClassifier()
Accuracy: 0.811015561015561
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      3715
           1       0.61      0.60      0.61      1169

    accuracy                           0.81      4884
   macro avg       0.74      0.74      0.74      4884
weighted avg       0.81      0.81      0.81      4884

Confusion Matrix:
[[3254  461]
 [ 462  707]]
--------------------------------------------------------------------------------
****************************************************************************************************
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
Model Name: Bagging
Fold # 9
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
***************************************************************************************

C:\Users\raqib\AppData\Local\Temp\ipykernel_15988\3388014965.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.fillna(X_train.mean(), inplace=True)
C:\Users\raqib\AppData\Local\Temp\ipykernel_15988\3388014965.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(X_train.mean(), inplace=True)
C:\Users\raqib\AppData\Local\Temp\ipykernel_15988\3388014965.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on 

Y after resampling: Counter({0: 30847, 1: 30847})
Model: BaggingClassifier()
Accuracy: 0.8075348075348076
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.87      0.87      3715
           1       0.59      0.62      0.61      1169

    accuracy                           0.81      4884
   macro avg       0.74      0.74      0.74      4884
weighted avg       0.81      0.81      0.81      4884

Confusion Matrix:
[[3214  501]
 [ 439  730]]
--------------------------------------------------------------------------------
****************************************************************************************************
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
Model Name: Bagging
Fold # 10
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
*************************************************************************************

C:\Users\raqib\AppData\Local\Temp\ipykernel_15988\3388014965.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.fillna(X_train.mean(), inplace=True)
C:\Users\raqib\AppData\Local\Temp\ipykernel_15988\3388014965.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(X_train.mean(), inplace=True)
C:\Users\raqib\AppData\Local\Temp\ipykernel_15988\3388014965.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on 

Y after resampling: Counter({0: 30877, 1: 30877})
Model: BaggingClassifier()
Accuracy: 0.7932022932022932
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.86      0.86      3715
           1       0.56      0.59      0.58      1169

    accuracy                           0.79      4884
   macro avg       0.72      0.73      0.72      4884
weighted avg       0.80      0.79      0.80      4884

Confusion Matrix:
[[3179  536]
 [ 474  695]]
--------------------------------------------------------------------------------
Average Accuracy for Bagging: 0.8075019133770412


All the accuracies:
****************************************************************************************************
****************************************************************************************************
{
    "Logistic Regression": 0.6622581202212727,
    "Decision Tree": 0.7683551454124637,
    "Random Forest": 0.8364522720356907,
    "Support V

### Highest Accuracy

In [8]:
mypckg.highest_accuracy(model_accuracies)

The higest accuracy in this experiment:
LightGBM -->> 0.8593628391581308


('LightGBM', 0.8593628391581308)